# Coordinate transformations

## Motivation

In all fields of science we frequently encounter data that is represented in coordinates or coordinate systems that are not apt for certain operations or visualizations.
In these cases we may thus need to compute new coordinates based on one or multiple existing coordinates.
For simple cases this may just be done by hand.
Consider:

In [ ]:
import scipp as sc
x = sc.linspace(dim='x', unit='m', start=1., stop=55., num=100)
da = sc.DataArray(data=x*x, coords={'x':x})
da.plot(figsize=(4,3))

We may want to use $x^2$ instead of $x$ as a coordinate, do highlight the quadratic nature of our data:

In [ ]:
da2 = da.copy()
da2.coords['x_square'] = x * x
da2

While adding a new coordinate may often be done with a single line of code, the above example highlights the first shortcoming of this approach:
To actually visualize `da` using this new coordinate we must additionally rename the dimension:

In [ ]:
da2 = da2.rename_dims({'x':'x_square'})
da2.plot(figsize=(4,3))

Further complications are:
- The original coordinate is preserved and may get in the way in subsequent operations.
- Event-coordinates of binned data are not handled.
- Multi-step conversions with multiple inputs and multiple outputs may be required in practice.

To accommodate these reoccuring yet highly application-specific needs, scipp provides a generic mechanism for transforming coordinates.
This is described and examplified in the following.

## `transform_coords`

[sc.transform_coords](../generated/functions/scipp.transform_coords.rst#scipp.transform_coords) (also available as method of data arrays and dataset) is a tool for transforming one or more input coordinates into one or more output coordinates.
It automatically handles:
- Renaming of dimensions, if dimension-coordinates are transformed.
- Change of coordinates to attributes to avoid interference of coordinates consumed by the transformation in follow-up operations.
- Conversion of event-coordinates of binned data, if present.

### Basic example

We start by revisiting the example given in [Motivation](#Motivation).
The building blocks `transform_coords` operates on are functions with named parameters.
The parameter names define the names of the *input coordinates to consume*.
Let us define `x_square`, which will *consume* `x`:

In [ ]:
def x_square(x):
    return x * x

Next, we create a `dict`, mapping from an output coord name to a function that can create this coordinate.
The [sc.show_graph](../generated/functions/scipp.show_graph.rst#scipp.show_graph) helper is a convient tool for visualizing the coordinate transformation defined by such as mapping:

In [ ]:
graph = {'x^2':x_square}
sc.show_graph(graph)

Here, the `x` coordinate can be consumed by the `x_square` function, creating the `x^2` coordinate.
Note that the function name and coordinate are unrelated.
Next, we can call `transform_coords`.
Apart from the graph, we also pass a list of desired output coordinates, here simply `['x^2']`.
`transform_coords` returns a new (shallow-copied) data array with added coordinates:

In [ ]:
transformed = da.transform_coords(['x^2'], graph=graph)
transformed

Note how `x` is now an attribute, i.e., operations will not use it for alignment anymore.
This is important since it will allow for operations combining `transformated` with other data that may have matching `x^2` but not `x`.

### Example: Multi-step transform splitting and combining input coords

In [ ]:
import scipp as sc
import numpy as np
hour_steps = sc.arange(dim='timestamp', dtype='int64', unit='s', start=0, stop=3*24*60*60, step=60*60)
start = sc.scalar(np.datetime64('2021-01-01T17:00:00'))
datetime = start + hour_steps

nsite = 1000
ntime = len(datetime)
# Note that these points are *note* uniformely distributed on a shpere, this is NOT a good way to
# generate such points
site = sc.vectors(dims=['cartesian'], values=np.random.rand(nsite, 3)) - sc.vector(value=[.5,.5,.5])
site /= sc.norm(site)
site *= 6.371 * sc.Unit('km')
site
da = sc.DataArray(
    data=sc.array(dims=['cartesian','timestamp'], values=np.random.rand(nsite, ntime)),
    coords={'cartesian':site, 'timestamp':datetime}
)
da += 2. * (site.fields.z > 0. * sc.Unit('km')).astype('float64')  # more lightning strikes in norther hemisphere

phi0 = sc.atan2(y=site.fields.y, x=site.fields.x) - sc.to_unit(90.0 * sc.Unit('deg'), 'rad')
sin = sc.sin(phi0+sc.linspace(dim='timestamp', unit='rad', start=0, stop=6*np.pi, num=ntime))
(sin+1).plot()
da += 2*(sin+1)
da

In [ ]:
da.plot(projection='3d', positions=site)

In [ ]:
def lat_long(cartesian):
    x = cartesian.fields.x
    y = cartesian.fields.y
    z = cartesian.fields.z
    theta = sc.atan2(y=sc.sqrt(x*x+y*y), x=z)
    phi = sc.atan2(y=y, x=x)
    return {'lattitude':theta, 'longitude':phi}

def time(datetime):
    seconds_per_day = sc.scalar(24*60*60, unit='s/D')
    start_day = sc.scalar(start.value.astype('datetime64[D]'))
    start_day_in_seconds = sc.scalar(start_day.values.astype('datetime64[s]'))
    offset = datetime - start_day_in_seconds
    time = (offset % seconds_per_day).astype('float64')
    return time
    
def timezone_offset(longitude):
    long = sc.to_unit(longitude, unit='deg', copy=False)
    return (long / (15.0 * sc.Unit('deg/hour'))).astype('int64') + 12 * sc.Unit('hour')

def local_time(timestamp, timezone_offset):
    return time(sc.to_unit(timezone_offset, timestamp.unit) + timestamp)

In [ ]:
graph = {'timezone_offset':timezone_offset,
         ('lattitude','longitude'):lat_long,
         'time':local_time
        }
sc.show_graph(graph)

In [ ]:
da.transform_coords(['lattitude'], graph=graph)

In [ ]:
da.transform_coords(['lattitude', 'longitude'], graph=graph)

In [ ]:
out = da.transform_coords(['time', 'longitude', 'lattitude'], graph=graph)
out

In [ ]:
time_edges = sc.linspace(dim='time', unit='s', start=0, stop=24*60*60, num=6)
# TODO deg
lattitude = sc.linspace(dim='lattitude', unit='rad', start=0, stop=np.pi, num=13)
longitude = sc.linspace(dim='longitude', unit='rad', start=-np.pi, stop=np.pi, num=13)
sc.bin(out.flatten(to='dummy'), edges=[lattitude, longitude, time_edges]).transpose().plot(resolution=12)
#sc.histogram(out, bins=time_edges).plot()

In [ ]:
def date_and_time(datetime):
    seconds_per_day = sc.scalar(24*60*60, unit='s/D')
    start_day = sc.scalar(start.value.astype('datetime64[D]'))
    start_day_in_seconds = sc.scalar(start_day.values.astype('datetime64[s]'))
    offset = datetime - start_day_in_seconds
    time = (offset % seconds_per_day).astype('float64')
    #date = start_day + offset // seconds_per_day
    date = offset // seconds_per_day
    return {'date':date, 'time':time}

In [ ]:
graph = {('date', 'time'):date_and_time}
sc.show_graph(graph)